In [ ]:
import streamlit as st
import requests
import json
from pathlib import Path
from dotenv import load_dotenv
from typing import Optional, Dict, Any
import logging
import os

BACKEND_HOST = os.getenv('BACKEND_HOST', 'localhost')
BACKEND_PORT = os.getenv('BACKEND_PORT', '777')
BASE_URL = f'http://{BACKEND_HOST}:{BACKEND_PORT}'

BASE_URL

In [ ]:

API_VERSION = 'v1'
LOGIN_URL = f"{BASE_URL}/api/{API_VERSION}/users/login/access-token"

response = requests.post(
    LOGIN_URL,
    json={'username': 'admin', 'password': 'admin'},  # Send as JSON
    timeout=10
)

response_json = response.json()
token = response_json.get('access_token') 

In [ ]:
response_json

In [ ]:
import streamlit as st
import requests
import logging
import os

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Environment setup
BACKEND_HOST = 'localhost'
BACKEND_PORT = '777'
BASE_URL = f'http://{BACKEND_HOST}:{BACKEND_PORT}'
API_VERSION = 'v1'
LOGIN_URL = f"{BASE_URL}/api/{API_VERSION}/users/login/access-token"

try:
    # Send as form data
    response = requests.post(
        LOGIN_URL,
        data={
            "username": "admin",
            "password": "admin"
        },
        headers={
            'Content-Type': 'application/x-www-form-urlencoded',
            'Accept': 'application/json'
        },
        timeout=10
    )
    
    # Force raise for bad status codes
    response.raise_for_status()
    
    # Get the token directly since it's returned as a string
    token = response.text.strip('"')  # Remove quotes from the token string
    
    if token:
        logger.info("Successfully retrieved token")
        st.success("Authentication successful!")
        st.session_state['token'] = token
        
        # Display token (you might want to remove this in production)
        st.write("Token received (first 20 characters):", token[:20] + "...")
    else:
        logger.error("No token in response")
        st.error("Failed to get authentication token")
        
except requests.exceptions.ConnectionError as e:
    error_msg = f"Failed to connect to {LOGIN_URL}. Error: {str(e)}"
    logger.error(error_msg)
    st.error(error_msg)
    
except requests.exceptions.Timeout:
    error_msg = "Request timed out. Please try again."
    logger.error(error_msg)
    st.error(error_msg)
    
except requests.exceptions.RequestException as e:
    error_msg = f"Request failed: {str(e)}"
    logger.error(error_msg)
    st.error(error_msg)

# If you need to use the token for subsequent requests, you can access it from session state
if 'token' in st.session_state:
    # Example of how to use the token in subsequent requests
    headers = {
        'Authorization': f'Bearer {st.session_state["token"]}',
        'Content-Type': 'application/json'
    }
    # You can now use these headers for other API calls

In [ ]:
token

In [ ]:
token

In [ ]:
# Endpoints
CHAT_ENDPOINT = 'message'
THREAD_MESSAGES_ENDPOINT = 'thread-messages'

# Full URLs
USER_QUESTION_URL = f"{BASE_URL}/api/{API_VERSION}/chat/{CHAT_ENDPOINT}"

data = {
	"user_question": "How many border stations are there in South Africa?"
}

headers = {"Authorization": f"Bearer {token}"}
response = requests.post(
    USER_QUESTION_URL,
    data=data,
    headers={
        'Authorization': f'Bearer {token}'
    },
    timeout=10
)

In [ ]:
response

In [ ]:
import requests
import json
import logging
import time
from typing import Dict, Any
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Environment setup for local access
BACKEND_HOST = 'localhost'
BACKEND_PORT = '777'
BASE_URL = f'http://{BACKEND_HOST}:{BACKEND_PORT}'
API_VERSION = 'v1'

# API Endpoints
CHAT_ENDPOINT = 'message'
USER_QUESTION_URL = f"{BASE_URL}/api/{API_VERSION}/chat/{CHAT_ENDPOINT}"

def create_session() -> requests.Session:
    """Create a session with retry strategy"""
    session = requests.Session()
    
    retry_strategy = Retry(
        total=3,
        backoff_factor=0.5,
        status_forcelist=[500, 502, 503, 504],
        allowed_methods=["POST", "GET"]
    )
    
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    
    return session

def send_chat_message(token: str, question: str, max_retries: int = 3) -> Dict[str, Any]:
    """
    Send a chat message with retries.
    """
    session = create_session()
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }
    
    data = {
        "user_question": question
    }
    
    for attempt in range(max_retries):
        try:
            logger.info(f"Attempt {attempt + 1} of {max_retries}")
            logger.info(f"Sending request to: {USER_QUESTION_URL}")
            
            response = session.post(
                USER_QUESTION_URL,
                json=data,
                headers=headers,
                timeout=(5, 60)  # Increased timeout to 60 seconds
            )
            
            response.raise_for_status()
            
            try:
                result = response.json()
                logger.info("Successfully received response")
                return result
            except json.JSONDecodeError as e:
                logger.error(f"Failed to decode JSON. Response text: {response.text}")
                raise
                
        except requests.exceptions.Timeout:
            logger.warning(f"Attempt {attempt + 1} timed out")
            if attempt == max_retries - 1:
                raise
            time.sleep(2 ** attempt)  # Exponential backoff
            
        except requests.exceptions.RequestException as e:
            logger.error(f"Request failed: {str(e)}")
            if attempt == max_retries - 1:
                raise
            time.sleep(2 ** attempt)
            
    raise Exception("All retry attempts failed")

# Example usage
try:
    # First get the token
    session = create_session()
    auth_response = session.post(
        f"{BASE_URL}/api/{API_VERSION}/users/login/access-token",
        data={
            "username": "admin",
            "password": "admin"
        },
        headers={
            'Content-Type': 'application/x-www-form-urlencoded',
            'Accept': 'application/json'
        },
        timeout=(5, 10)
    )
    auth_response.raise_for_status()
    token = auth_response.text.strip('"')
    
    # Send the chat message
    question = "How many border stations are there in South Africa?"
    logger.info(f"Sending question: {question}")
    
    response_data = send_chat_message(token, question)
    print("\nResponse:", json.dumps(response_data, indent=2))
    
except requests.exceptions.RequestException as e:
    print(f"\nError occurred: {str(e)}")
    logger.exception("Request failed")

finally:
    session.close()

In [ ]:
session = create_session()
auth_response = session.post(
    f"{BASE_URL}/api/{API_VERSION}/users/login/access-token",
    data={
        "username": "admin",
        "password": "admin"
    })
auth_response.raise_for_status()
token = auth_response.text.strip('"')

In [ ]:
token


In [ ]:
import requests
import json
import logging
import time
from typing import Dict, Any
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Environment setup for local access
BACKEND_HOST = 'localhost'
BACKEND_PORT = '777'
BASE_URL = f'http://{BACKEND_HOST}:{BACKEND_PORT}'
API_VERSION = 'v1'

# API Endpoints
CHAT_ENDPOINT = 'message'
USER_QUESTION_URL = f"{BASE_URL}/api/{API_VERSION}/chat/{CHAT_ENDPOINT}"

def create_session() -> requests.Session:
    """Create a session with retry strategy"""
    session = requests.Session()
    
    retry_strategy = Retry(
        total=3,
        backoff_factor=0.5,
        status_forcelist=[500, 502, 503, 504],
        allowed_methods=["POST", "GET"]
    )
    
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount("http://", adapter)
    session.mount("https://", adapter)
    
    return session


In [ ]:
GET_THREADS_URL

In [ ]:
# Environment setup for local access
BACKEND_HOST = 'localhost'
USER_THREADS = 'user-threads'
BACKEND_PORT = '777'
BASE_URL = f'http://{BACKEND_HOST}:{BACKEND_PORT}'
API_VERSION = 'v1'
THREAD_MESSAGES = 'thread-messages'

# API Endpoints
CHAT_ENDPOINT = 'message'
USER_QUESTION_URL = f"{BASE_URL}/api/{API_VERSION}/chat/{CHAT_ENDPOINT}"
GET_USER_THREADS_URL = f"{BASE_URL}/api/{API_VERSION}/chat/{USER_THREADS}?user_id=1"
LOGIN_URL = f"{BASE_URL}/api/{API_VERSION}/users/login/access-token"
GET_THREAD_MESSAGES_URL = f"{BASE_URL}/api/{API_VERSION}/chat/{THREAD_MESSAGES}?thread_id=thread_YpNLFDjyRnSHxsQaezCAM5hK"
# f"{{base_url}}/api/v1/chat/thread-messages" #

In [ ]:
response = requests.post(
    LOGIN_URL,
    data={
        "username": "admin",
        "password": "admin"
    },
    headers={
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': 'application/json'
    },
    timeout=10
)

# Force raise for bad status codes
response.raise_for_status()

# Get the token directly since it's returned as a string
token = response.text.strip('"')  # Remove quotes from the token string
print(token)

In [ ]:
    session = create_session()
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }

In [ ]:
response = session.get(
    GET_USER_THREADS_URL,
    # json=data,
    headers=headers,
    timeout=(5, 60)  # Increased timeout to 60 seconds
)
        

response.json()

In [ ]:
GET_THREAD_MESSAGES_URL

In [ ]:
response = session.get(
    GET_THREAD_MESSAGES_URL,
    # json=data,
    headers=headers,
    timeout=(5, 60)  # Increased timeout to 60 seconds
)
        

response.json()

In [ ]:
thread_jzXBjAtOTf28bKbwvOBsMQJy